In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import hashlib
import random
import os
import pickle
import joblib
os.chdir('Resources/')

In [2]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [3]:
def string_to_int(message):
    return int.from_bytes(message.encode('utf-8'), byteorder='big')

def int_to_string(value):
    byte_length = (value.bit_length() + 7) // 8 
    return value.to_bytes(byte_length, byteorder='big').decode('utf-8', errors='ignore')

def float_to_string(message):
    return str(message)

def string_to_float(message):
    return float(message)

In [4]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [5]:
def mod_inv(a, m):
    origin_m = m
    y, x = 0, 1

    if m == 1:
        return 0

    while a > 1:
        q = a // m
        temp = m

        m = a % m
        a = temp
        temp = y

        y = x - q * y
        x = temp

    if x < 0:
        x += origin_m
    return x

In [6]:
df = pd.read_csv('20_(SYS)_CC_Received_User_Data.csv')
#df = df.dropna()
df1 = df.iloc[:, :5]
df2 = df.iloc[:, 5:10]
df3 = df.iloc[:, 10:]  
df3

,ST_Slope,HeartDisease
0,Flat,NaN


In [7]:
def encrypt_data(n, df, public_keys, column_names):
    encrypted_data = pd.DataFrame()

    for k in range(n):
        public_key = public_keys[column_names[k]] 
        p = public_key['p']
        g = public_key['g']
        h = public_key['h']
        y = public_key['y']
            
        print(f"Processing Column: {column_names[k]}")

        s = power(h, y, p)

        messages = df.iloc[:, k].values 

        encrypted_column = []

        for i, message in enumerate(messages, 1):
            if message is not None and message == message:  
                print(f"Processing Message {i}: {message}")
                
                temp_message = string_to_int(message)  
                
                c1 = power(g, y, p)
                c2 = (temp_message * s) % p
                
                encrypted_column.append(f"({c1},{c2})") 
                print(f"Ciphertext: (c1, c2): ({c1}, {c2})")
            
            else:
                encrypted_column.append(None)

        encrypted_data[column_names[k]] = encrypted_column

    return encrypted_data

In [8]:
public_keys = load_key("5_1_CR1_Public_Key_Part.pkl")

column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol"
]

encrypted_data_1 = encrypt_data(5, df1, public_keys, column_names)
encrypted_data_1

Processing Column: Age
Processing Message 1: Middle-aged
Ciphertext: (c1, c2): (7270988666945721113120809463312144500555280693674, 29378076229834480526623698013398561105293578426979)
Processing Column: Sex
Processing Message 1: Male
Ciphertext: (c1, c2): (22961665442842143903220898566890700943449852505536, 52767206925905853065827991660681686067837424862391)
Processing Column: ChestPainType
Processing Message 1: Atypical Angina
Ciphertext: (c1, c2): (20613262322062277119573630365746595127182499134519, 10626643683754479036384694824612448404945809430203)
Processing Column: RestingBloodPressure
Processing Message 1: High
Ciphertext: (c1, c2): (25666449378145533821257044298909866450131224188388, 44129450815403147156592275346933750510317032563972)
Processing Column: Cholesterol
Processing Message 1: High
Ciphertext: (c1, c2): (49648777350264920336257917599386766751429992537949, 1135939455600870294993318531954353030495459122925)


,Age,Sex,ChestPainType,RestingBloodPressure,Cholesterol
0,(727098866694572111312080946331214450055528069...,(229616654428421439032208985668907009434498525...,(206132623220622771195736303657465951271824991...,(256664493781455338212570442989098664501312241...,(496487773502649203362579175993867667514299925...


In [9]:
public_keys = load_key("5_2_CR2_Public_Key_Part.pkl")

column_names = [
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina", "Oldpeak"
]

encrypted_data_2 = encrypt_data(5, df2, public_keys, column_names)
encrypted_data_2

Processing Column: FastingBloodSugar
Processing Message 1: Yes
Ciphertext: (c1, c2): (23262138535430815965897209751063602705729030703607, 4914346123643601822726979785757666703777986672995)
Processing Column: RestingECG
Processing Message 1: Left Ventricular Hypertrophy
Ciphertext: (c1, c2): (20663312364214574545779485666796513311973587596483, 48486076259671370764155465307622813075662573960886)
Processing Column: MaximumHeartRate
Processing Message 1: High
Ciphertext: (c1, c2): (52792865455644029778549137754509322291183002304751, 28983986677276752523453900194013762641905265375780)
Processing Column: ExerciseAngina
Processing Message 1: Yes
Ciphertext: (c1, c2): (53567916807806914103914877537850847808169443022861, 28334333845225812059600031434937154322565505902319)
Processing Column: Oldpeak
Processing Message 1: Mild
Ciphertext: (c1, c2): (22453991272873058234492300158377019032521111612219, 9272096283892224631377319923711877207820123633085)


,FastingBloodSugar,RestingECG,MaximumHeartRate,ExerciseAngina,Oldpeak
0,(232621385354308159658972097510636027057290307...,(206633123642145745457794856667965133119735875...,(527928654556440297785491377545093222911830023...,(535679168078069141039148775378508478081694430...,(224539912728730582344923001583770190325211116...


In [10]:
public_keys = load_key("5_3_CC_Public_Key_Part.pkl")

column_names = [
    "ST_Slope"
]

encrypted_data_3 = encrypt_data(1, df3, public_keys, column_names)
encrypted_data_3

Processing Column: ST_Slope
Processing Message 1: Flat
Ciphertext: (c1, c2): (10235051201618776623875359480548401671315373635138, 39327948452405280433016459820259504149041535047652)


,ST_Slope
0,(102350512016187766238753594805484016713153736...


In [11]:
encrypted_df = pd.concat([encrypted_data_1, encrypted_data_2, encrypted_data_3], axis='columns')
encrypted_df
encrypted_df.to_csv('21_(SYS)_CR_Encrypted_User_Data.csv', index=False)